# Technical Factor - Market Synchronization Index (MSI)

In [21]:
import dai
import pandas as pd

In [22]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [23]:
sql = f"""
WITH
data_factor AS (
    WITH
    data_stock AS (
        SELECT
            date,
            instrument,
            open AS security_open,
            close AS security_close
        FROM cn_stock_bar1d
    ),
    data_index AS (
        SELECT
            date,
            open AS index_open,
            close AS index_close
        FROM cn_stock_index_bar1d
        WHERE instrument = '000300.SH'
    ),
    data_merge AS (
        SELECT
            date,
            instrument,
            security_open,
            security_close,
            index_open,
            index_close,
            IF((security_close >= security_open AND index_close >= index_open) OR (security_close < security_open AND index_close < index_open), 1, 0) AS SYNC,
            m_avg(IF((security_close >= security_open AND index_close >= index_open) OR (security_close < security_open AND index_close < index_open), 1, 0), 6) AS MSI
        FROM data_stock
        JOIN data_index USING (date)
    )
    SELECT
        date,
        instrument,
        MSI,
        SYNC
    FROM data_merge
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(MSI > 0.5 AND m_lag(MSI, 1) <= 0.5, 1, 0) AS TRBY1,
        IF(MSI < 0.5 AND m_lag(MSI, 1) >= 0.5, 1, 0) AS TRSL1
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(MSI > 0.5 AND (MSI - m_lag(MSI, 1)) > 0, 1, 0) AS MTBY1,
        IF(MSI < 0.5 AND (MSI - m_lag(MSI, 1)) < 0, 1, 0) AS MTSL1
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(MSI < 0.3 AND (MSI - m_lag(MSI, 1)) > 0, 1, 0) AS RVBY1,
        IF(MSI > 0.7 AND (MSI - m_lag(MSI, 1)) < 0, 1, 0) AS RVSL1
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(MSI, 1) <= 0.7 AND MSI > 0.7, 1, 0) AS BKBY1,
        IF(m_lag(MSI, 1) >= 0.3 AND MSI < 0.3, 1, 0) AS BKSL1
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        MSI,
        SYNC,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
AND instrument IN {str(tuple(instrument_list))}
ORDER BY date, instrument
"""

In [24]:
df = dai.query(sql, filters={"date":[sd,ed]}).df()
df

,date,instrument,MSI,SYNC,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-09,600519.SH,0.833333,1,0,0,0,0,0,0,0,0
1,2025-01-10,600519.SH,0.833333,1,0,0,0,0,0,0,0,0
2,2025-01-13,600519.SH,0.833333,1,0,0,0,0,0,0,0,0
3,2025-01-14,600519.SH,0.833333,1,0,0,0,0,0,0,0,0
4,2025-01-15,600519.SH,0.833333,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
233,2025-12-25,600519.SH,0.500000,1,0,0,0,0,0,0,0,0
234,2025-12-26,600519.SH,0.333333,0,0,1,0,1,0,0,0,0
235,2025-12-29,600519.SH,0.333333,1,0,0,0,0,0,0,0,0
236,2025-12-30,600519.SH,0.333333,0,0,0,0,0,0,0,0,0
